# Linear Algebra and Optimization for Machine Learning - Project

In [44]:
# imports
import numpy as np
import pandas as pd

# set seed
np.random.seed(42)

### 1.

(a) Generate a $300 \times 20$ data matrix $X$, where each entry is uniformly random.  
Generate an outcome vector $y$, which is a linear combination of the columns of $X$ with uniformly random weights, and some Gaussian noise added to each entry of $y$.

In [45]:
# shape
m = 300
n = 20

# feature matrix, Xij ~ Unif([0, 1]), X.shape = (300, 20)
X = np.random.uniform(size = (m, n))

# generate random noise eps, eps.shape = (300,)
eps = np.random.normal(size = m)

# uniformly random weights weight, weight.shape = (20,)
weight = np.random.uniform(size = n)

# y = X weight + eps, y.shape = (300,)
y = X @ weight + eps

# print shape of the data
print("The shape of the data is: ", X.shape)

The shape of the data is:  (300, 20)


(b) Write a function to divide the data set into a train and test set.

In [46]:
# function for splitting into training and testing datasets and option for validation set
def train_test_split(X, y, p_train: float, p_val: float = None):
    '''
    splits feature matrix and response vector according to given split percentage
    either splits into training and testing set (if p_val = None)
    or training, validation, and testing set (if p_val != None)
    
    (X, y): datapoints
    p_train: fraction of data that is in training set --> round up
    p_val: fraction of data that is in validation set --> round up
    
    returns: X_train, (X_val), X_test, y_train, (y_val), y_test
    '''

    # shape of feature matrix
    m = X.shape[0]
    n = X.shape[1]

    # calculate split index for training set
    split_train = int(np.ceil(p_train * m))

    # split feature and response according to split index
    # check if validation set is wanted
    if p_val is None:
        # just training and testing
        X_train, X_test = X[:split_train], X[split_train:]
        y_train, y_test = y[:split_train], y[split_train:]
        return X_train, X_test, y_train, y_test
    else:
        # calculate validation split index
        split_val = int(np.ceil((split_train + p_val * m)))
        X_train, X_val, X_test = X[:split_train], X[split_train:split_val], X[split_val:]
        y_train, y_val, y_test = y[:split_train], y[split_train:split_val], y[split_val:]
        return X_train, X_val, X_test, y_train, y_val, y_test

In [47]:
# example usage
p = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, p)

# with validation set
p_train = 0.7
p_val = 0.15

X_train_v, X_val_v, X_test_v, y_train_v, y_val_v, y_test_v = train_test_split(X, y, p_train, p_val)

(c) Write functions for OLS and Ridge regression and apply this to your synthetic data set. Discuss the performance on train and test sets.

In [48]:
# calculate MSE of prediction
def evaluate(X, y, w):
    '''
    evaluates the performance (MSE) of some weights matrix w for features X and response y
    assumes linear relationship --> Xw = y
    
    (X, y): datapoints
    w: weights vector

    returns: MSE
    '''
    MSE = np.mean((X @ w - y)**2)
    return MSE

# OLS solution using normal equations
# use Moore-Penrose inverse --> if the inverse exists, then X(-1) == X(+)
def OLS(X_train, y_train):
    '''
    gives least-norm OLS solution for the given feature matrix and corresponding response vector
    uses the moore-penrose pseudo-inverse where w = X+ y
    when the inverse exists, X+ = X-1

    (X_train, y_train): datapoints

    returns: weights vector w
    '''
    # solve using the pseudo-inverse (least-norm OLS solution)
    w = np.linalg.pinv(X_train) @ y_train

    return w
    

# Ridge Regression
def ridge(X_train, y_train, lamb):
    '''
    gives solution to ridge regression for the given feature matrix and corresponding response vector with regularization term lamb

    (X_train, y_train): datapoints
    lamb: regularization parameter

    returns: weights vector w
    '''
    # number of datapoints
    n = X_train.shape[1]

    # unique minimizer for Ridge Regression
    w = np.linalg.inv(X_train.T @ X_train + lamb * np.eye(n)) @ X_train.T @ y_train

    return w


# find optimal lambda using validation set
def find_opt_lam(X_train, y_train, X_val, y_val):
    '''
    finds optimal value for lambda for ridge regression
    uses training set to find w for a given lambda
    uses validation set to compare performance of w
    
    (X_train, y_train): datapoints in training set
    (X_val, y_val): datapoints in validation set

    returns: opt_lam
    '''
    # initialize performance and optimal lambda values
    min_perf = float('inf')
    opt_lam = None

    # determine performance on validation set and update opt_lam if performance is better
    for lamb in np.arange(0.01, 100, 0.01):
        w_ridge = ridge(X_train, y_train, lamb)
        perf_val = evaluate(X_val, y_val, w_ridge)

        if perf_val < min_perf:
            min_perf = perf_val
            opt_lam = lamb

    return opt_lam

In [49]:
# example usage
# use training, validation, test split to keep comparisons fair

# OLS solution
w_OLS = OLS(X_train_v, y_train_v)

# Ridge solution
# find optimal lambda using training and validation set
opt_lam = find_opt_lam(X_train_v, y_train_v, X_test_v, y_test_v)

# solve ridge
w_ridge = ridge(X_train_v, y_train_v, opt_lam)

# performance
perf_OLS_tr, perf_OLS_te = evaluate(X_train_v, y_train_v, w_OLS), evaluate(X_test_v, y_test_v, w_OLS)
perf_ridge_tr, perf_ridge_te = evaluate(X_train_v, y_train_v, w_ridge), evaluate(X_test_v, y_test_v, w_ridge)

# print performance
print(f"OLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")

OLS train MSE: 0.8847
OLS test MSE: 1.5692
-----------------------------------
ridge train MSE: 0.9029
ridge test MSE: 1.5474


### Question 1 (c) — Performance Discussion

We compare **Ordinary Least Squares (OLS)** and **Ridge Regression** on the synthetic dataset.

| Model | Train MSE | Test MSE | Observation |
|:--|:--:|:--:|:--|
| **OLS** | 0.8847 | 1.5692 | Low train error but higher test error → slight overfitting |
| **Ridge** | 0.9029 | 1.5474 | Slightly higher train error, lower test error → better generalization |

The OLS model achieves the lowest error on the training set, showing it fits the data well.  

However, its test MSE is higher, meaning it captures some noise from the training data and thus overfits slightly.  

Ridge regression introduces an ℓ₂-penalty that shrinks the coefficients, trading a bit of bias for reduced variance.  

As expected, the training MSE increases a little (0.8847 → 0.9029), but the test MSE decreases (1.5692 → 1.5474).  

This demonstrates the bias–variance trade-off: by constraining the model complexity, Ridge achieves better generalization on unseen data.  

The improvement is modest, which suggests the dataset is not strongly affected by multicollinearity or noise, yet regularization still provides a small stability gain.


(d) Create a data matrix with many multicolinearities by adding a large number (say, 200) columns to X that are linear combinations of the original 20 columns with some Gaussian noise added to each entry. Run OLS and Ridge regression and discuss the performance on train and test sets. Is it hard to find a good value for $\lambda$?

In [50]:
def add_multicolinearity(X, num):
    '''
    add multicolinearity to feature matrix X by adding (num) columns that are linear combinations of the original columns
    weights for linear combinations are uniformly random
    gaussian noise added to each entry

    X: feature matrix
    num: number of new columns to be added

    returns: X_new
    '''
    # number of entries, features
    n_entries = X.shape[0]
    n_feats = X.shape[1]

    # initialize new columns list
    new_cols = []
    
    for i in range(num):
        # uniformly random weights w, w.shape = (n_feats, )
        w = np.random.uniform(size = n_feats)

        # generate random noise eps, eps.shape = (n_entries,)
        eps = np.random.normal(size = n_entries)

        # linear combination of features + noise
        X_col = X @ w + eps
        new_cols.append(X_col.reshape(-1, 1))

    # concatenate the X with new columns
    X_new = np.hstack([X] + new_cols)
    return X_new

In [51]:
# add multicolinearity
X_new = add_multicolinearity(X, 200)
print("The multicolinearity data shape is: ", X_new.shape)

# split with validation set
p_train = 0.7
p_val = 0.15
X_train_new, X_val_new, X_test_new, y_train_new, y_val_new, y_test_new = train_test_split(X_new, y, p_train, p_val)

# find optimal lambda
opt_lam = find_opt_lam(X_train_new, y_train_new, X_val_new, y_val_new)
print("\nThe optimal lambda is: ", np.round(opt_lam, 4))

# example usage
w_OLS = OLS(X_train_new, y_train_new)
w_ridge = ridge(X_train_new, y_train_new, opt_lam)

# performance
perf_OLS_tr, perf_OLS_te = evaluate(X_train_new, y_train_new, w_OLS), evaluate(X_test_new, y_test_new, w_OLS)
perf_ridge_tr, perf_ridge_te = evaluate(X_train_new, y_train_new, w_ridge), evaluate(X_test_new, y_test_new, w_ridge)

# print performace
print(f"\nOLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")

The multicolinearity data shape is:  (300, 220)

The optimal lambda is:  99.99

OLS train MSE: 0.0
OLS test MSE: 21.9828
-----------------------------------
ridge train MSE: 0.2979
ridge test MSE: 1.7687


### Question 1 (d) — Performance Discussion

After adding 200 multicollinear columns, the dataset became highly redundant. The OLS model has a train MSE of 0.0 and a test MSE of 21.9828. The test error is an order of magnitude higher, which shows that the model is affected by the multicollinearity.

In contrast, ridge regression performed much better, with a train MSE of 0.2979 and a test MSE of 1.7687. The L2 regularization term stabilizes the solution by shrinking correlated coefficients, reducing variance, and preventing overfitting to noise. The chosen lambda value of approximately 100 shows that strong regularization was required to counteract the effects of multicollinearity.

(e) Now instead of adding multicolinearities, add many superficial feature columns to X which have no relation to the outcome vector y. Again run OLS and Ridge regression and discuss the performance on train and test sets.

In [52]:
def add_superficial(X, num):
    '''
    adds (num) superficial features to feature matrix X --> have no relation to response vector

    X: feature matrix
    num: number of superficial features to add

    returns: X_new
    '''
    # number of entries
    n_entries = X.shape[0]

    # uniformly random features
    sup_feats = np.random.uniform(size = (n_entries, num))

    # concatenate the X with new columns
    X_new = np.hstack([X, sup_feats])
    return X_new

In [53]:
# add superficial features
X_new = add_superficial(X, 200)
print("The superficial data shape is: ", X_new.shape)

# split with validation set
p_train = 0.7
p_val = 0.15
X_train_new, X_val_new, X_test_new, y_train_new, y_val_new, y_test_new = train_test_split(X_new, y, p_train, p_val)

# find optimal lambda
opt_lam = find_opt_lam(X_train_new, y_train_new, X_val_new, y_val_new)
print("\nThe optimal lambda is: ", round(opt_lam, 4))

# example usage
w_OLS = OLS(X_train_new, y_train_new)
w_ridge = ridge(X_train_new, y_train_new, opt_lam)

# performance
perf_OLS_tr, perf_OLS_te = evaluate(X_train_new, y_train_new, w_OLS), evaluate(X_test_new, y_test_new, w_OLS)
perf_ridge_tr, perf_ridge_te = evaluate(X_train_new, y_train_new, w_ridge), evaluate(X_test_new, y_test_new, w_ridge)

print(f"\nOLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")

The superficial data shape is:  (300, 220)

The optimal lambda is:  43.07

OLS train MSE: 0.0
OLS test MSE: 30.2725
-----------------------------------
ridge train MSE: 0.7491
ridge test MSE: 2.1213


### Question 1 (e) — Performance Discussion

When adding many superficial features unrelated to the outcome, we get a similar effect as before. The OLS model again has a train MSE of 0.0 and a test MSE of 30.2725. The order of magnitude between train and test error indicates that OLS overfits the noise introduced by the irrelevant features, resulting in poor generalization.

Ridge regression handled the situation much better, with a train MSE of 0.7491 and a test MSE of 2.1213. The regularization term penalized large coefficients and effectively reduced the influence of irrelevant variables. The optimal lambda of around 13.03 suggests that moderate regularization was sufficient to control overfitting.

## 2.

(a) Implement functions for logistic regression and hinge-loss classification.

In [54]:
# new evals function since they have a different model
def evaluate_logistic(X, y, w):
    '''
    evaluates the performance of logistic regression and returns the mean squared error
    '''
    y_pred = logistic_predict(X, w)
    mse = np.mean((y - y_pred)**2)
    return mse

def evaluate_hinge(X, y, w):
    '''
    evaluates the performance of hinge-loss and returns the mean squared error
    '''
    y_pred = hinge_predict(X, w)
    mse = np.mean((y - y_pred)**2)
    return mse

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def logistic_predict(X, w, threshold=0.5):
    p = sigmoid(X @ w) # this is the probability of the data point being in class 1 (so between 0 and 1)
    return np.where(p >= threshold, 1, -1)

def hinge_predict(X, w):
    return np.where(X @ w >= 0, 1, -1)

def train_logistic_gd(X, y, lam, lr=0.01, num_iter=1000):
    '''
    train the logistic regression model and returns the weight vector
    '''
    n, d = X.shape
    w = np.zeros(d)

    for i in range(num_iter):
        p = sigmoid(-y * (X @ w))
        grad_w = -(X.T @ (p * y)) / n + lam * w
        w -= lr * grad_w
    return w

def train_hinge_gd(X, y, lam, C=1.0, lr=0.01, num_iter=1000):
    '''
    train the hinge-loss model and returns the weight vector
    '''
    n, d = X.shape
    w = np.zeros(d)

    for i in range(num_iter):
        scores = X @ w
        viol = y * scores < 1

        grad_w = w - C * (X[viol].T @ y[viol]) + lam * w

        # update
        w -= lr * grad_w
    return w


(b) Create a random data matrix $X$ and construct an output vector $y$ by generating and a random weight vector $w$ and setting $y_i = \text{sign}(x^T_i w)$, where $x^T_i$ is the $i$-th row of $X$. Use a test/train split and check the performance of OLS, Ridge regression, logistic regression and hinge-loss classification for binary classification. Do you see a large difference in performance between these methods?

In [55]:
# create a random data matrix X
n = 300
d = 20

X = np.random.normal(size = (n, d))
w = np.random.normal(size = d)
y = np.sign(X @ w)

p = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, p)

# Set lambda
lam = 0.01

# perform OLS
w_OLS = OLS(X_train, y_train)
perf_OLS_tr, perf_OLS_te = evaluate(X_train, y_train, w_OLS), evaluate(X_test, y_test, w_OLS)

# perform Ridge
w_ridge = ridge(X_train, y_train, lam)
perf_ridge_tr, perf_ridge_te = evaluate(X_train, y_train, w_ridge), evaluate(X_test, y_test, w_ridge)

# perform logistic regression
w_log = train_logistic_gd(X_train, y_train, lam)
perf_log_tr, perf_log_te = evaluate_logistic(X_train, y_train, w_log), evaluate_logistic(X_test, y_test, w_log)

# perform hinge loss
w_hinge = train_hinge_gd(X_train, y_train, lam)
perf_hinge_tr, perf_hinge_te = evaluate_hinge(X_train, y_train, w_hinge), evaluate_hinge(X_test, y_test, w_hinge)

# print all performances
print(f"OLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")
print("-" * 35)
print(f"logistic train MSE: {np.round(perf_log_tr, 4)}")
print(f"logistic test MSE: {np.round(perf_log_te, 4)}")
print("-" * 35)
print(f"hinge train MSE: {np.round(perf_hinge_tr, 4)}")
print(f"hinge test MSE: {np.round(perf_hinge_te, 4)}")

OLS train MSE: 0.3514
OLS test MSE: 0.3624
-----------------------------------
ridge train MSE: 0.3514
ridge test MSE: 0.3624
-----------------------------------
logistic train MSE: 0.0952
logistic test MSE: 0.3111
-----------------------------------
hinge train MSE: 0.0381
hinge test MSE: 0.0889


(c) Now create a data set $(X, y)$ for binary classification (with $X \in \mathbb{R}^{n \times d}$ and $y \in \{−1, 1\}^n$) such that, given a test/train split, OLS and Ridge perform very badly but logistic regression and hinge-loss classification perform well. What kind of properties of your data set are responsible for this?

In [56]:
# create a random data matrix X
n = 300
d = 20

X1 = np.random.normal(loc=1, scale=1, size = (250, d))
X2 = np.random.normal(loc=100, scale=0.1, size =(50, d))
X = np.concatenate([X1, X2])
np.random.shuffle(X)
w = np.random.uniform(low=-0.05, high=0.95, size = d) # Uniform distribution which is heavily skewed to being positive
y = np.sign(X @ w)

p = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, p)

# Set lambda
lam = 0.01

# perform OLS
w_OLS = OLS(X_train, y_train)
perf_OLS_tr, perf_OLS_te = evaluate(X_train, y_train, w_OLS), evaluate(X_test, y_test, w_OLS)

# perform Ridge
w_ridge = ridge(X_train, y_train, lam)
perf_ridge_tr, perf_ridge_te = evaluate(X_train, y_train, w_ridge), evaluate(X_test, y_test, w_ridge)

# perform logistic regression
w_log = train_logistic_gd(X_train, y_train, lam)
perf_log_tr, perf_log_te = evaluate_logistic(X_train, y_train, w_log), evaluate_logistic(X_test, y_test, w_log)

# perform hinge loss
w_hinge = train_hinge_gd(X_train, y_train, lam)
perf_hinge_tr, perf_hinge_te = evaluate_hinge(X_train, y_train, w_hinge), evaluate_hinge(X_test, y_test, w_hinge)

# print all performances
print(f"OLS train MSE: {np.round(perf_OLS_tr, 4)}")
print(f"OLS test MSE: {np.round(perf_OLS_te, 4)}")
print("-" * 35)
print(f"ridge train MSE: {np.round(perf_ridge_tr, 4)}")
print(f"ridge test MSE: {np.round(perf_ridge_te, 4)}")
print("-" * 35)
print(f"logistic train MSE: {np.round(perf_log_tr, 4)}")
print(f"logistic test MSE: {np.round(perf_log_te, 4)}")
print("-" * 35)
print(f"hinge train MSE: {np.round(perf_hinge_tr, 4)}")
print(f"hinge test MSE: {np.round(perf_hinge_te, 4)}")

OLS train MSE: 0.687
OLS test MSE: 0.8228
-----------------------------------
ridge train MSE: 0.687
ridge test MSE: 0.8228
-----------------------------------
logistic train MSE: 0.0
logistic test MSE: 0.0
-----------------------------------
hinge train MSE: 0.0
hinge test MSE: 0.0


### Question 2 (c) — Discussion
The data we generated has unbalanced classes, this is a result of the w vector and each x_i being chosen from a distribution which is heavily skewed to being positive. Thus, when we generate the y vector with the same formula used in 2 (b) most of the data points will be classified in the same class. The data also has many points which very clearly belong to a particular class, this comes from the 'X2' points which are narrowly distributed very positive points when compared to the overall distribution.